In [1]:
import cv2
import numpy as np
import os
import gc
import matplotlib.pyplot as pyp
import sklearn.cluster as skl

import time

In [2]:
# Initialize global variables and enable garbage collection
# This is gonna be a huge job
gc.enable()
dir = "/home/alice/Jupyter/OpenCV/images/"
out = "/home/alice/Jupyter/OpenCV/output/"
buoy_dir = "/home/alice/Jupyter/OpenCV/Buoys/"

buoy = cv2.imread(buoy_dir+"redbuoy.png")
buoy = cv2.cvtColor(buoy, cv2.COLOR_BGR2GRAY)
img = []
original = []
max_size = len(os.listdir(dir))
iteration = 0
arr_size = 500

In [3]:
# Read in images and pass them to the transform function
def iterator():
    global max_size
    for i in range(0,max_size):
        img.append(cv2.imread(dir + "left" + str(i).zfill(4) + ".jpg"))
        original.append(img[i % arr_size])
        
        transform(i)
    return True

In [4]:
#Process image by deleting background information, blurring, doing a sobel transform, and thresholding
#Most stable version for quickly generating a dataset
def transform_stable(i):
    global iteration
    global original
    if i > 0 and (len(img) % arr_size == 0 or len(img) + arr_size * iteration == max_size):        
        for j in range(0, max_size):
            im = np.array(img[j])[0:500][0:800]
            im = delete_blue(im)
            im = np.uint8(im)
            im = cv2.cvtColor(im, cv2.COLOR_HSV2BGR)
            im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            
            write(im, j)
            
        reset()
        iteration = iteration + 1
        return True
    return False

In [5]:
#Process image by deleting background information, blurring, doing a sobel transform, and thresholding
#In-progress function, test new functionality here
def transform(i):
    global iteration
    global original
    if i > 0 and (len(img) % arr_size == 0 or len(img) + arr_size * iteration == max_size):        
        for j in range(0, len(img)):
            im = np.array(img[j])[0:500][0:800]
            im = delete_blue(im)
            im = cv2.cvtColor(im, cv2.COLOR_HSV2BGR)
            im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            
            im = cv2.GaussianBlur(im, ksize=(0,0), sigmaX=0.5, sigmaY=.75, borderType=cv2.BORDER_DEFAULT)
            im = cv2.threshold(im, 40, 255, cv2.THRESH_BINARY)[1]
            im = cv2.medianBlur(im, 3)
            im = cv2.GaussianBlur(im, ksize=(0,0), sigmaX=0.5, sigmaY=.75, borderType=cv2.BORDER_DEFAULT)
            
            im_nz = np.nonzero(im)
            im_nz = np.column_stack((im_nz[1], im_nz[0]))
            
            cluster(im_nz, j)
            
            write(im, j)
            
        reset()
        iteration = iteration + 1
        return True
    return False


In [6]:
def cluster(image, index):    
    global iteration
    try:
        clustering = skl.DBSCAN(eps = 1, min_samples = 3, n_jobs=-1)
        clustering = clustering.fit(image)
        clustering = clustering.labels_
    except:
        print("image %d has no clusters" % (index + iteration*arr_size))
    
    #print(clustering)
    
    return False
    

In [7]:
#Delete all pixels that are outside of a certain red/green range
def delete_blue(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10)))] = [0,0,0]
    return(image)

In [8]:
#for writing one instance of an image
def write(image, index):
    cv2.imwrite(out + "out" + str(index + iteration * arr_size).zfill(4) + ".jpg", image)
    return True

In [9]:
#for writing multiple instances of an image. ex. when separating an image into red and green channels
def write_rg(image, index, color_index):
    cv2.imwrite(out + "out" + str(index + iteration * arr_size).zfill(4) + "-" + str(color_index) + ".jpg", image)
    return True

In [10]:
#an extension of write_rg for multiple sets of images. ex. recording multiple filters of red and green channels of an image
def write_rg_mult(image, index, color_index, mult):
    cv2.imwrite(out + "out" + str(index + iteration * arr_size).zfill(4) + "-" + str(color_index) + "-" + str(mult) + ".jpg", image)
    return True

In [11]:
#Delete the contents of the image arrays, used for preserving memory by doing operations on images in batches
def reset():
    del img[:]
    del original[:]
    return True

In [12]:
if __name__ == "__main__":
    reset()
    start_time =time.time()
    iterator()
    print('% seconds', time.time()-start_time )

% seconds 51.73242926597595
